In [1]:
import pandas as pd
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

### **Dataset: Monthly_data_cmo_filtered.csv**

#### **Data preprocessing for Time Series Analysis** 

Combining the APMC and Commodity to define unique clusters 

In [2]:
df = pd.read_csv("Monthly_data_cmo_filtered.csv")
df["date"] = pd.to_datetime(df["date"])
df["quarter"]=df["date"].dt.quarter
df["combination_label"]=df["APMC"]+":"+df["Commodity"]
df.shape

(62225, 13)

In [3]:
df.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name,quarter,combination_label
0,Ahmednagar,bajri,2014,September,24,1400,1750,1575.0,2014-09-01,Ahmadnagar,Maharashtra,3,Ahmednagar:bajri
1,Ahmednagar,bajri,2014,November,395,1507,1750,1629.0,2014-11-01,Ahmadnagar,Maharashtra,4,Ahmednagar:bajri
2,Ahmednagar,bajri,2015,January,97,1472,1764,1618.0,2015-01-01,Ahmadnagar,Maharashtra,1,Ahmednagar:bajri
3,Ahmednagar,bajri,2015,February,10,1500,1800,1650.0,2015-02-01,Ahmadnagar,Maharashtra,1,Ahmednagar:bajri
4,Ahmednagar,bajri,2015,March,14,1350,1700,1525.0,2015-03-01,Ahmadnagar,Maharashtra,1,Ahmednagar:bajri


In [4]:
data=df[["APMC","Commodity","modal_price","Year","combination_label"]].groupby(["APMC","Commodity","Year","combination_label"],as_index=False).count().rename(columns={"modal_price":"Count"}).reset_index(drop=True)
data.head() #Count total number of APMC & Commodity clusters appearing in the dataset from January to December
#data["Count"].max()
#data["Count"].min()

,APMC,Commodity,Year,combination_label,Count
0,Aamgaon,paddy-unhusked,2014,Aamgaon:paddy-unhusked,4
1,Aamgaon,paddy-unhusked,2015,Aamgaon:paddy-unhusked,12
2,Aamgaon,paddy-unhusked,2016,Aamgaon:paddy-unhusked,11
3,Aarni,black gram,2015,Aarni:black gram,1
4,Aarni,black gram,2016,Aarni:black gram,2


**Removing the clusters which are having Count less than 12. i.e. having the datapoints for atleast a year**

In [5]:
data = data[data["Count"]==12].reset_index(drop=True)
data.head()

,APMC,Commodity,Year,combination_label,Count
0,Aamgaon,paddy-unhusked,2015,Aamgaon:paddy-unhusked,12
1,Akhadabalapur,soybean,2015,Akhadabalapur:soybean,12
2,Akluj,bitter gourd,2015,Akluj:bitter gourd,12
3,Akluj,brinjal,2015,Akluj:brinjal,12
4,Akluj,cabbage,2015,Akluj:cabbage,12


In [6]:
filtered = data["combination_label"].unique().tolist()
df_filtered = df[df["combination_label"].isin(filtered)].reset_index(drop=True)
df_filtered.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name,quarter,combination_label
0,Akole,bajri,2014,September,25,1467,1733,1650.0,2014-09-01,Ahmadnagar,Maharashtra,3,Akole:bajri
1,Akole,bajri,2014,October,30,1390,1700,1560.0,2014-10-01,Ahmadnagar,Maharashtra,4,Akole:bajri
2,Akole,bajri,2014,November,67,1400,1700,1550.0,2014-11-01,Ahmadnagar,Maharashtra,4,Akole:bajri
3,Akole,bajri,2014,December,40,1375,1750,1563.0,2014-12-01,Ahmadnagar,Maharashtra,4,Akole:bajri
4,Akole,bajri,2015,January,74,1395,1840,1580.0,2015-01-01,Ahmadnagar,Maharashtra,1,Akole:bajri


In [7]:
df_filtered.to_csv("seasonal_data_analysis.csv",index=False)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25027 entries, 0 to 25026
Data columns (total 13 columns):
APMC                 25027 non-null object
Commodity            25027 non-null object
Year                 25027 non-null int64
Month                25027 non-null object
arrivals_in_qtl      25027 non-null int64
min_price            25027 non-null int64
max_price            25027 non-null int64
modal_price          25027 non-null float64
date                 25027 non-null datetime64[ns]
district_name        25027 non-null object
state_name           25027 non-null object
quarter              25027 non-null int64
combination_label    25027 non-null object
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 2.5+ MB


### **Task - 2.1** 

#### **Detect seasonality type (multiplicative or additive) for each cluster of APMC and commodities**

**Additive Model**
-> An additive model suggests that the components are added together as follows:
   y(t) = Level + Trend + Seasonality + Noise
-> An additive model is linear where changes are made constantly by the same        amount

**Multiplicative Model**
-> An multiplicative model suggests that the components are multiplied together    as follows:
   y(t) = Level * Trend * Seasonality * Noise
-> An mulitplicative model is nonlinear such as quadratic or expotential where      changes are increases or decreases overtime.

In additive model, the amplitude and frequency of the cycles are same where in multiplicative model, the amplitude and frequency varies.

Using seasonal decompose for decomposing the data and Auto Correlation function for determining the seasonal type of the cluster

In [23]:
def check_seasonality_type(data):
    data["Period"] = data["Year"].astype(str) + data["Month"]
    data['Period'] = pd.to_datetime(data['Period'], format="%Y%B")
    
    #Dataframe to perform decomposition on
    df = pd.DataFrame(data=data['modal_price'].values, index=data["Period"],columns=['modal_price'])
    df.index=pd.to_datetime(df.index)
    
    #Decomposing data
    
    try:
        resultadd = seasonal_decompose(df,model='additive',freq=12)
        resultmult = seasonal_decompose(df, model="multiplicative",freq=12)
        
        additive_acf = sum(np.asarray(acf(resultadd.resid, missing='drop'))*2)
        multiplicative_acf = sum(np.asarray(acf(resultmult.resid, missing='drop'))*2)

    except:
        #reducing the frequency if insufficient data points
        resultadd = seasonal_decompose(df,model='additive',freq=6)
        resultmult = seasonal_decompose(df, model="multiplicative",freq=6)
        
        additive_acf = sum(np.asarray(acf(resultadd.resid, missing='drop'))*2)
        multiplicative_acf = sum(np.asarray(acf(resultmult.resid, missing='drop'))*2)
        
        
    if additive_acf<multiplicative_acf:
        return "Additive"
    else:
        return "Multiplicative"

#### **Dataset: seasonal_data_analysis.csv**

In [16]:
df=pd.read_csv("seasonal_data_analysis.csv")
df["date"]=pd.to_datetime(df["date"])
df.shape

(25027, 13)

In [24]:
commodities=list(df["combination_label"].unique())
seasonality_type=[]

# commodity here stands for combination of APMC and commodity
for commodity in commodities:
    data_filter=df[df["combination_label"]==commodity]
    seasonality_type.append((commodity,check_seasonality_type(data_filter)))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:350: RuntimeWarning: invalid value encountered in double_scalars
  xo = x - x.sum()/notmask_int.sum()
C:\Users\DELL\Anaconda3\lib\site-packages\statsmodels\tsa\stattool

In [25]:
# Storing Seasonality types (Task 2.1 completed)

dataframe=pd.DataFrame(seasonality_type,columns=["Commodity","Seasonality Type"])
dataframe.to_csv("seasonality_type.csv",index=False)
dataframe.head()

,Commodity,Seasonality Type
0,Akole:bajri,Multiplicative
1,Akole:paddy-unhusked,Additive
2,Akole:wheat(husked),Multiplicative
3,Akole:gram,Additive
4,Akole:soybean,Multiplicative


### **Task - 2.2**

#### **De-seasonalise prices for each commodity and APMC according to the detected seasonality type**

For de-seasonalise the modal_prices:

-> *if additive* - subtract the seasonal value from the modal_price

-> *if multiplicative* - divide the seasonal value from the modal_price


In [10]:
def deseasonalize_values(data,value):
    data["Period"] = data["Year"].astype(str) + data["Month"]
    data['Period'] = pd.to_datetime(data['Period'], format="%Y%B")
    
    #Dataframe to perform decomposition on
    df = pd.DataFrame(data=data['modal_price'].values, index=data["Period"],columns=['modal_price'])
    df.index=pd.to_datetime(df.index)
    
    #Decomposing data
    
    try:
        resultadd = seasonal_decompose(df,model='additive',freq=12)
        resultmult = seasonal_decompose(df, model="multiplicative",freq=12)
        
    except:
        #reducing the frequency if insufficient data points
        resultadd = seasonal_decompose(df,model='additive',freq=6)
        resultmult = seasonal_decompose(df, model="multiplicative",freq=6)
        
    if value=="Additive":
        df['Seasonal'] = resultadd.seasonal.values
        
        #removing seasonality component from additive data
        values = df["modal_price"] - df["Seasonal"]
        values = values.tolist()
        
        indices = data.index.tolist()
        values_range=range(0,len(values))
        for index, value in zip(indices,values_range):
            data.at[index,"deseasonalise_price"]=values[value]
        
        return data[["APMC","Commodity","combination_label","date","modal_price","quarter","deseasonalise_price"]]
    
    else:
        df['Seasonal'] = resultmult.seasonal.values
        #removing seasonality component from additive data
        values = df["modal_price"]/df["Seasonal"]
        values = values.tolist()
        
        indices = data.index.tolist()
        values_range=range(0,len(values))
        for index, value in zip(indices,values_range):
            data.at[index,"deseasonalise_price"]=values[value]
        
        return data[["APMC","Commodity","combination_label","date","modal_price","quarter","deseasonalise_price"]]
        

In [8]:
df_type=pd.read_csv("seasonality_type.csv").set_index("Commodity").to_dict()["Seasonality Type"]
df_data=pd.read_csv("seasonal_data_analysis.csv")
df_data["deseasonalise_price"]=np.nan #initialize empty column

In [13]:
commodities = list(df_type.keys())
data_main = pd.DataFrame()
for commodity in commodities:
    data_temp=deseasonalize_values(df_data[df_data["combination_label"]==commodity],df_type[commodity])
    data_main=pd.concat([data_main,data_temp])
data_main.to_csv("deseasonalize_data.csv",index=False)

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
data_main.head()

,APMC,Commodity,combination_label,date,modal_price,quarter,deseasonalise_price
0,Akole,bajri,Akole:bajri,2014-09-01,1650.0,3,1664.590398
1,Akole,bajri,Akole:bajri,2014-10-01,1560.0,4,1588.085197
2,Akole,bajri,Akole:bajri,2014-11-01,1550.0,4,1572.191403
3,Akole,bajri,Akole:bajri,2014-12-01,1563.0,4,1536.385799
4,Akole,bajri,Akole:bajri,2015-01-01,1580.0,1,1607.065791
